In [19]:
# Import pandas & numpy & regular expression
import pandas as pd
import numpy as np
import re
# Sklearn has a helper that makes it easy to do cross validation
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
# Import the logistic regression class
from sklearn.linear_model import LogisticRegression
# Import the random forest model class
from sklearn.ensemble import RandomForestClassifier
# Import the gradient boosting classifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
train_df = pd.read_csv('data/train.csv', header=0)
#train_df[0:5]
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 647054 entries, 0 to 647053
Data columns (total 7 columns):
TripType                 647054 non-null int64
VisitNumber              647054 non-null int64
Weekday                  647054 non-null object
Upc                      642925 non-null float64
ScanCount                647054 non-null int64
DepartmentDescription    645693 non-null object
FinelineNumber           642925 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 39.5+ MB
None


In [40]:
def print_unique(df, list):
    for column in list:
        print("\n Totally there are " + str(len(df[column].unique())) + "  " +df[column].name)
        
#print_unique(train_df,["VisitNumber","TripType", "DepartmentDescription", "FinelineNumber",'Upc'])

In [41]:
# add missing columns in test if needed
def find_diff_list (listA, listB):
    result = []
    index = []
    if len(listA) > len(listB):
        print("first parameter size larger")
        for item in listA:
            if  not (item in listB):
                print(listA.index(item))
                result.append(item)
    else:
        print("second parameter size larger")
        for item in listB:
            if  not (item in listA):
                print(listB.index(item))
                result.append(item)
    return result

In [42]:
test = list(test_df.DepartmentDescription.unique())
train = list(train_df.DepartmentDescription.unique())
print(find_diff_list(train, test))
#print(find_diff_list(test, train))

first parameter size larger
9
68
[nan, 'HEALTH AND BEAUTY AIDS']


In [43]:
def build_df(df, train):
    #fillna
    df.DepartmentDescription = df.DepartmentDescription.fillna('nan')
    
    #generate lists for index, vistors, depts, types, weekdays
    vistors = list(df.VisitNumber.unique())
    depts = list(df.DepartmentDescription.unique())
    if train != 1:
        depts.insert(68,'HEALTH AND BEAUTY AIDS')
    depts.sort()

    
    def category_mapping_name (list):
        result = []
        i = 0
        for item in list:
            result.append("dep"+str(i))
            i += 1
        return result
    
    if train == 1:
        types = list(df.TripType.unique())
        types.sort()
        types_str = []
        for type in types:
            types_str.append("TripType" + str(type))
        
    weekdays = list(df.Weekday.unique())
    
    dept_matrix = np.zeros((len(vistors),len(depts)))
    print("\n #######Number of dept : " + str(dept_matrix.shape))
    if train == 1:
        trip_type = np.zeros((len(vistors),len(types)))
        print("\n #######Number of trips : " + str(trip_type.shape))
    
    count = np.zeros((len(set(vistors)),1))
    print("\n #######Number of count : " + str(count.shape))
    weekday = np.zeros((len(set(vistors)),7))
    print("\n #######Number of weekday : " + str(weekday.shape))
    
    #initilization
    i=0
    j=0
    dept_matrix[j,depts.index(df.DepartmentDescription[i])] = df.ScanCount[i]
    if train == 1:
        trip_type[j,types.index(df.TripType[i])] = 1
    count[j] = 1
    weekday[j, weekdays.index(df.Weekday[i])] = 1
    
    #generating matrix
    for i in range(1,len(df)):
        if df.VisitNumber[i] != df.VisitNumber[i-1]:
            j = j + 1
            #print(j)
            if train == 1:
                trip_type[j,types.index(df.TripType[i])] = 1
            count[j] = 1
        elif ((train == 1) and (df.TripType[i] != df.TripType[i-1])):
            print("ERROR! Different trip type assigned for same visit!")
            count_[j] += 1
        else:
            count[j] += 1
        dept_matrix[j,depts.index(df.DepartmentDescription[i])] += df.ScanCount[i]
        weekday[j,weekdays.index(df.Weekday[i])] = 1
    
    #merge
    if train == 1:
        dept_weekday_trip_matrix = np.hstack((weekday, dept_matrix, trip_type))
    else:
        dept_weekday_trip_matrix = np.hstack((weekday, dept_matrix))
    
    print("\n #######Dimension : " + str(dept_weekday_trip_matrix.shape))
    weekday_2 = weekdays.copy()
    #weekday_2.extend(depts) here using mapping after insert and sort
    weekday_2.extend(category_mapping_name(depts))
    
    if train == 1:
        weekday_2.extend(types_str)
    
    dept_weekday_trip_matrix_df = pd.DataFrame(dept_weekday_trip_matrix, columns = weekday_2)
    #dept_weekday_trip_matrix_df.head()
    return dept_weekday_trip_matrix_df

## (1) train and test transform

In [44]:
train_trans_df = build_df(train_df , 1)
train_trans_df.head()


 #######Number of dept : (95674, 69)

 #######Number of trips : (95674, 38)

 #######Number of count : (95674, 1)

 #######Number of weekday : (95674, 7)

 #######Dimension : (95674, 114)


In [63]:
train_trans_df.head()
print(list(train_trans_df.columns)[0:76])
print(list(train_trans_df.columns)[76:])

['Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'dep0', 'dep1', 'dep2', 'dep3', 'dep4', 'dep5', 'dep6', 'dep7', 'dep8', 'dep9', 'dep10', 'dep11', 'dep12', 'dep13', 'dep14', 'dep15', 'dep16', 'dep17', 'dep18', 'dep19', 'dep20', 'dep21', 'dep22', 'dep23', 'dep24', 'dep25', 'dep26', 'dep27', 'dep28', 'dep29', 'dep30', 'dep31', 'dep32', 'dep33', 'dep34', 'dep35', 'dep36', 'dep37', 'dep38', 'dep39', 'dep40', 'dep41', 'dep42', 'dep43', 'dep44', 'dep45', 'dep46', 'dep47', 'dep48', 'dep49', 'dep50', 'dep51', 'dep52', 'dep53', 'dep54', 'dep55', 'dep56', 'dep57', 'dep58', 'dep59', 'dep60', 'dep61', 'dep62', 'dep63', 'dep64', 'dep65', 'dep66', 'dep67', 'dep68']
['TripType3', 'TripType4', 'TripType5', 'TripType6', 'TripType7', 'TripType8', 'TripType9', 'TripType12', 'TripType14', 'TripType15', 'TripType18', 'TripType19', 'TripType20', 'TripType21', 'TripType22', 'TripType23', 'TripType24', 'TripType25', 'TripType26', 'TripType27', 'TripType28', 'TripType29', 'TripTyp

In [54]:
train_trans_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95674 entries, 0 to 95673
Columns: 114 entries, Friday to TripType999
dtypes: float64(114)
memory usage: 83.9 MB


In [46]:
test_df = pd.read_csv('data/test.csv', header=0)

In [47]:
test_trans_df = build_df(test_df , 0)
test_trans_df.head()


 #######Number of dept : (95674, 69)

 #######Number of count : (95674, 1)

 #######Number of weekday : (95674, 7)

 #######Dimension : (95674, 76)


,Friday,Saturday,Sunday,Monday,Tuesday,Wednesday,Thursday,dep0,dep1,dep2,...,dep59,dep60,dep61,dep62,dep63,dep64,dep65,dep66,dep67,dep68
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
#print(test_trans_df.columns)
#print(train_trans_df.columns[0:76])

## (2) Prepare xgboosting

training data processed
Train xgboost model


In [120]:
np.arange(0, len(train_trans_df))
features_train = list(train_trans_df.columns)[0:76]
output_train = list(train_trans_df.columns)[76:]

print(train_trans_df[output_train][0:1]["TripType999"] == 1)
TripTypeAll_list = list(train_df.TripType.unique())
TripTypeAll_list.sort()
print(TripTypeAll_list)

print(train_trans_df[output_train][0:0+1]['TripType'+str(999)] == 1)
print(train_trans_df[output_train][0:0+1]['TripType'+str(999)].any() == 1)

0    True
Name: TripType999, dtype: bool
[999, 30, 26, 8, 35, 41, 21, 6, 42, 7, 9, 39, 25, 38, 15, 36, 20, 37, 32, 40, 5, 3, 4, 24, 33, 43, 31, 27, 34, 18, 29, 44, 19, 23, 22, 28, 14, 12]
0    True
Name: TripType999, dtype: bool
True


In [124]:
#Convert back list to one number and using only [0:76]
def merge_df_columns(df):
    list = []
    for i in np.arange(0, df.shape[0]):
        for j in TripTypeAll_list:
            if df[i:i+1]['TripType'+str(j)].any() == 1:
                #print(j)
                list.append(j)
    series_out = pd.Series(list)
    return series_out

In [149]:
TripTypeAll_list.sort()
print(TripTypeAll_list)

[3, 4, 5, 6, 7, 8, 9, 12, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 999]


In [150]:
# expand Series to dataframe
def expand_df(df , TripTypeAll_list, name):
    out_matrix = np.zeros((len(df),len(TripTypeAll_list)))
    for i in np.arange(0, len(df)):
        for j in np.arange(0, len(TripTypeAll_list)):
            if df[i].any() == TripTypeAll_list[j]:
                out_matrix [i][j] = 1
            else:
                out_matrix [i][j] = 0
    types_str = []
    for type in TripTypeAll_list:
        types_str.append("TripType" + str(type))
    output_df = pd.DataFrame(out_matrix,columns = types_str)
    return output_df    

In [125]:
merged_TripType = merge_df_columns(train_trans_df[output_train])
#merged_TripType

0        999
1         30
2         26
3          8
4          8
5         35
6         41
7         21
8          6
9         42
10         7
11         9
12         8
13        39
14        25
15         8
16         9
17         8
18         8
19         8
20         8
21         8
22         8
23        38
24         8
25        35
26        15
27         9
28         8
29         8
        ... 
95644     21
95645     44
95646      8
95647     44
95648     40
95649     26
95650      8
95651      8
95652     36
95653     40
95654     42
95655     36
95656     31
95657      9
95658     42
95659     30
95660      7
95661     38
95662     40
95663     38
95664     24
95665      9
95666     32
95667     38
95668      8
95669     25
95670     22
95671     39
95672     39
95673      8
dtype: int64

In [135]:
print('training data processed')

def rmspe(y, yhat):
    return np.sqrt(np.mean(((y - yhat)/y) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y, yhat)

print("Train xgboost model")

params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.1,
          "max_depth": 10,
          "subsample": 0.85,
          "colsample_bytree": 0.4,
          "min_child_weight": 6,
          "silent": 1,
          "thread": 1,
          "seed": 1301
          }
num_boost_round = 10000

training data processed
Train xgboost model


In [136]:
print("Train a XGBoost model")
#X_train, X_valid = train_test_split(train_trans_df, test_size=0.12, random_state=10)
split_train = int(len(train_trans_df)*0.7)
print(split_train)
X_train = train_trans_df[0:split_train]
X_valid = train_trans_df[split_train:]
features_train = list(train_trans_df.columns)[0:76]
#output_train = list(train_trans_df.columns)[76]
#print(X_train[features_train])
dtrain = xgb.DMatrix(X_train[features_train],merged_TripType[0:split_train])
dvalid = xgb.DMatrix(X_valid[features_train],merged_TripType[split_train:])

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, \
  early_stopping_rounds=500, verbose_eval=True)
# , feval=rmspe_xg

Will train until eval error hasn't decreased in 500 rounds.
[0]	train-rmse:291.536469	eval-rmse:289.729736
[1]	train-rmse:283.874847	eval-rmse:282.147125
[2]	train-rmse:275.038757	eval-rmse:273.292053
[3]	train-rmse:267.377136	eval-rmse:265.414215
[4]	train-rmse:262.283234	eval-rmse:259.976105
[5]	train-rmse:257.895599	eval-rmse:255.367752
[6]	train-rmse:253.490891	eval-rmse:250.797775
[7]	train-rmse:248.619537	eval-rmse:245.954971
[8]	train-rmse:245.129318	eval-rmse:242.304520
[9]	train-rmse:242.180786	eval-rmse:239.180542
[10]	train-rmse:237.789398	eval-rmse:234.842178
[11]	train-rmse:233.982239	eval-rmse:230.963577
[12]	train-rmse:231.855530	eval-rmse:228.771591
[13]	train-rmse:229.697586	eval-rmse:226.626404
[14]	train-rmse:227.869278	eval-rmse:224.846634
[15]	train-rmse:226.382462	eval-rmse:223.388565
[16]	train-rmse:224.883987	eval-rmse:221.991180
[17]	train-rmse:223.476807	eval-rmse:220.601807
[18]	train-rmse:221.900742	eval-rmse:219.140640
[19]	train-rmse:220.670929	eval-rmse:2

Train a XGBoost model
66971


[3543]	train-rmse:159.130997	eval-rmse:175.431885
Stopping. Best iteration:
[3043]	train-rmse:161.462234	eval-rmse:175.312714



In [138]:
def rmse_cal (A, B):
    return np.sqrt(np.mean(A-B)**2)

In [139]:
print("Validating")
yhat = gbm.predict(xgb.DMatrix(X_valid[features_train]))
error = rmse_cal(merged_TripType[split_train:], yhat)
print('RMSE: {:.6f}'.format(error))



Validating
RMSE: 0.108201


In [147]:
output_index = pd.Series(test_df.VisitNumber.unique())

In [151]:
print("Make predictions on the test set")
dtest = xgb.DMatrix(test_trans_df)
test_probs = gbm.predict(dtest)

Make predictions on the test set


In [153]:
print(len(test_probs))
test_probs

95674


array([ 271.20458984,   26.68092728,  328.31188965, ...,   83.31121063,
        -59.46408081,  -68.52149963], dtype=float32)

In [ ]:
# Make Submission
expand_df(test_probs, TripTypeAll_list, 'TripType')
result = pd.DataFrame({"VisitNumber": output_index, 'Sales': np.expm1(test_probs)})
result.to_csv("xgboost_10_submission.csv", index=False)

## test date read in and transform

In [74]:
prediction_20000 = alg.predict(dept_weekday_matrix_test[0:20000])

In [75]:
prediction_40000 = alg.predict(dept_weekday_matrix_test[20000:40000])

In [76]:
prediction_60000 = alg.predict(dept_weekday_matrix_test[40000:60000])

In [77]:
prediction_80000 = alg.predict(dept_weekday_matrix_test[60000:80000])

In [78]:
prediction_90000 = alg.predict(dept_weekday_matrix_test[80000:])

In [79]:
prediction = np.concatenate((prediction_20000, prediction_40000, prediction_60000, prediction_80000, prediction_90000))

In [80]:
prediction.shape

(95674, 38)

In [81]:
prediction_list = prediction.tolist()

## submission format

In [82]:
sample_sub_df = pd.read_csv('data/sample_submission.csv', header=0)

In [83]:
sample_sub_df.columns

Index(['VisitNumber', 'TripType_3', 'TripType_4', 'TripType_5', 'TripType_6',
       'TripType_7', 'TripType_8', 'TripType_9', 'TripType_12', 'TripType_14',
       'TripType_15', 'TripType_18', 'TripType_19', 'TripType_20',
       'TripType_21', 'TripType_22', 'TripType_23', 'TripType_24',
       'TripType_25', 'TripType_26', 'TripType_27', 'TripType_28',
       'TripType_29', 'TripType_30', 'TripType_31', 'TripType_32',
       'TripType_33', 'TripType_34', 'TripType_35', 'TripType_36',
       'TripType_37', 'TripType_38', 'TripType_39', 'TripType_40',
       'TripType_41', 'TripType_42', 'TripType_43', 'TripType_44',
       'TripType_999'],
      dtype='object')

In [84]:
types.sort()
print(types)
types_str = []
for type in types:
    types_str.append("TripType_" + str(type))
print(types_str)
print(test_df["VisitNumber"].unique()[0:10])

[3, 4, 5, 6, 7, 8, 9, 12, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 999]
['TripType_3', 'TripType_4', 'TripType_5', 'TripType_6', 'TripType_7', 'TripType_8', 'TripType_9', 'TripType_12', 'TripType_14', 'TripType_15', 'TripType_18', 'TripType_19', 'TripType_20', 'TripType_21', 'TripType_22', 'TripType_23', 'TripType_24', 'TripType_25', 'TripType_26', 'TripType_27', 'TripType_28', 'TripType_29', 'TripType_30', 'TripType_31', 'TripType_32', 'TripType_33', 'TripType_34', 'TripType_35', 'TripType_36', 'TripType_37', 'TripType_38', 'TripType_39', 'TripType_40', 'TripType_41', 'TripType_42', 'TripType_43', 'TripType_44', 'TripType_999']
[ 1  2  3  4  6 13 14 16 18 21]


In [85]:
output_df = pd.DataFrame(prediction_list, columns = types)
output_df_2 = output_df.reindex_axis(sorted(output_df.columns), axis=1)
output_df_2.columns = types_str
output_df_2.insert(0, "VisitNumber", test_df["VisitNumber"].unique())
output_df_2["VisitNumber"] = test_df["VisitNumber"].unique()
output_df_2[0:5]

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
output_df_2.to_csv("submission_2.csv", index=False)